# TCGA - Breast Cancer Classification
# TABLE OF CONTENTS

1. [INTRODUCTION](#1)

   - [Problem Description](#1.1)
   - [Scope of the Project](#1.2)
   - [Goals](#1.3)

2. [THE DATA](#2)

   - [Data Sources](#2.1)
   - [Data Collection](#2.2)
   - [Feature Description Table](#2.3) 

3. [NORMALIZATION](#3)
 
4. [EXPLORATORY DATA ANALYSIS (EDA)](#4)




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# 1. INTRODUCTION <a id='1'></a>

## 1.1 Problem Description <a id='1.1'></a>

Breast cancer remains one of the most prevalent and life-threatening diseases affecting individuals worldwide. This study aims to classify breast cancer stages by examining gene expression variations and predicting patient survival likelihood using transcriptomic data. By leveraging mRNA-seq gene expression profiles from The Cancer Genome Atlas (TCGA), the project will identify key biomarkers for early detection and explore gene-gene interactions influencing cancer progression. A combination of normalization techniques, feature selection methods, and machine learning models will be employed to develop an optimized classification system, with a final goal of integrating survival analysis into a predictive dashboard for clinical insights.

## 1.2 Scope of the Project <a name="1.2"></a>

1. Data Collection: Profile the mRNA-seq gene expression data and clinical information for breast cancer samples from TCGA, integrating them into a single data frame.
2. Data Preprocessing: Explore 3 normalization techniques for gene expression data: RPKM, FPKM, and TPM. After normalization, perform EDA to better understand the data distribution, outliers, and key characteristics, focusing on the behavior of genes across different cancer stages.
3. Feature Selection & Feature Engineering: Apply dimensionality reduction techniques (e.g., PCA, LDA, PCC) for feature selection and investigate transcription factor activities for model input.
4. Model Selection & Training: Train multiple models to predict cancer severity/stage (e.g., Ridge, RF, MLP, XGBoost, and SVM) and compare their performance using cross-validation.
5. Model Evaluation & Optimization: Evaluate models using performance metrics (e.g., accuracy, AUC, F1-score) and optimize the selected model using techniques like hyperparameter tuning or grid search to improve performance.
6. Create a Survival Analysis Dashboard: Develop a survival analysis dashboard leveraging Gradio/Tableau for the interface, PostgreSQL for data storage, and Apache Airflow to schedule and run the ML model for predicting survival probabilities of breast cancer patients.

## 2.2 Goals <a name="2.2"></a>

The goal of this study is to:

- Examine the differences in gene expression across various stages of breast cancer.
- Predict the likelihood of survival for breast cancer patients based on gene expression.
- Identify key biomarkers for the early detection of breast cancer.
- Explore gene-gene interactions and their role in cancer progression.


# 2. THE DATA<a id='2'></a>

## 2.1 Data Sources<a id='2.1'></a>

The dataset is sourced from TCGA human tumor data (transcriptomes) and is focused solely on breast cancer (BRCA) samples from cancer patients. We will be using mRNA-seq gene expression profiles of TCGA breast tumors, which are freely available in the TCGA database in TSV format. It consists of 1,231 breast tissue samples from 1,095 unique patients, with each sample containing 60,660 gene expression measurements. These measurements represent mRNA-seq counts, where higher counts indicate greater gene expression levels.

## 2.3 Data Collection <a name="2.2"></a>

- The TCGA-STAD_RNAseq_sample_info.tsv dataset consists of 97 features and 1,231 observations, providing detailed sample metadata. 

- The TCGA-STAD_RNAseq_sample_assay.tsv dataset contains 60,660 features and 1,231 observations, representing gene expression values across different samples. 

- After merging both datasets using barcodes, the resulting dataset expands to 1,231 observations and 60,757 features. 

- A preliminary check confirms that no duplicate entries and missing values are present in the dataset.

In [18]:
# Load datasets
info_df = pd.read_csv('data/TCGA-STAD_RNAseq_sample_info.tsv', sep='\t')
assay_df = pd.read_csv('data/TCGA-STAD_RNAseq_sample_assay.tsv', sep='\t')

print("\nInfo Dataset:")
display(info_df.head())

print("\nAssay Dataset:")
display(assay_df.head())


Info Dataset:


,barcode,patient,sample,shortLetterCode,definition,sample_submitter_id,sample_type_id,tumor_descriptor,sample_id,submitter_id,...,paper_tobacco_smoking_history,paper_CNV.Clusters,paper_Mutation.Clusters,paper_DNA.Methylation.Clusters,paper_mRNA.Clusters,paper_miRNA.Clusters,paper_lncRNA.Clusters,paper_Protein.Clusters,paper_PARADIGM.Clusters,paper_Pan.Gyn.Clusters
0,TCGA-A2-A25D-01A-12R-A16F-07,TCGA-A2-A25D,TCGA-A2-A25D-01A,TP,Primary solid Tumor,TCGA-A2-A25D-01A,1,Primary,eda0ff5c-84cf-44cb-888f-0aab17441dfa,TCGA-A2-A25D,...,NaN,C6,C4,C1,C2,C2,C2,C1,C4,C3
1,TCGA-BH-A201-01A-11R-A14M-07,TCGA-BH-A201,TCGA-BH-A201-01A,TP,Primary solid Tumor,TCGA-BH-A201-01A,1,Primary,7893162b-3d19-4c54-86ee-aa4884d84e8a,TCGA-BH-A201,...,NaN,C5,C4,C1,C2,C3,C2,C1,C6,C1
2,TCGA-AC-A23C-01A-12R-A169-07,TCGA-AC-A23C,TCGA-AC-A23C-01A,TP,Primary solid Tumor,TCGA-AC-A23C-01A,1,Primary,9cbd4bd6-244b-4274-a230-a7bd02e79571,TCGA-AC-A23C,...,NaN,C3,C1,C1,C1,C3,C1,C1,C6,C3
3,TCGA-AR-A5QP-01A-11R-A28M-07,TCGA-AR-A5QP,TCGA-AR-A5QP-01A,TP,Primary solid Tumor,TCGA-AR-A5QP-01A,1,Primary,151a1187-e07e-4b76-b2d2-2da133b9116b,TCGA-AR-A5QP,...,NaN,C5,C7,C1,C2,C2,NaN,C2,C6,C1
4,TCGA-C8-A12P-01A-11R-A115-07,TCGA-C8-A12P,TCGA-C8-A12P-01A,TP,Primary solid Tumor,TCGA-C8-A12P-01A,1,Primary,0d5aa7a3-3488-4339-b8f1-e4971912e2bc,TCGA-C8-A12P,...,NaN,C4,C9,C2,C2,C3,C1,C5,C4,C4



Assay Dataset:


,TCGA-A2-A25D-01A-12R-A16F-07,TCGA-BH-A201-01A-11R-A14M-07,TCGA-AC-A23C-01A-12R-A169-07,TCGA-AR-A5QP-01A-11R-A28M-07,TCGA-C8-A12P-01A-11R-A115-07,TCGA-BH-A0W3-01A-11R-A109-07,TCGA-AC-A8OQ-01A-11R-A41B-07,TCGA-BH-A0H6-01A-21R-A056-07,TCGA-D8-A27V-01A-12R-A17B-07,TCGA-D8-A27M-01A-11R-A16F-07,...,TCGA-AC-A3TM-01A-11R-A22K-07,TCGA-A7-A26J-01A-11R-A277-07,TCGA-LL-A441-01A-11R-A24H-07,TCGA-A7-A13G-01A-11R-A13Q-07,TCGA-BH-A0E6-01A-11R-A034-07,TCGA-LL-A5YP-01A-21R-A28M-07,TCGA-AO-A03L-01A-41R-A056-07,TCGA-BH-A42T-01A-11R-A24H-07,TCGA-A2-A04W-01A-31R-A115-07,TCGA-AR-A24O-01A-11R-A169-07
0,4474,3555,731,3960,3834,826,690,5534,13509,2496,...,1153,599,5172,3299,7389,2300,955,2641,2718,1856
1,9,30,113,50,0,8,55,29,15,39,...,5,0,68,24,20,7,3,40,1,58
2,1785,2537,5947,2155,3161,1483,2456,1511,3060,1446,...,1118,2843,4333,1465,1366,2524,3231,2311,1914,2184
3,1362,1865,3118,1355,1411,1594,646,929,7099,1702,...,573,2677,2584,5199,650,720,1763,1263,677,1206
4,433,890,1063,474,637,645,764,300,1449,655,...,281,930,971,1257,1414,632,899,904,291,574


In [19]:
# Basic Dataset Information
print("\nDataset Information:")
info_df.info()
assay_df.info()


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231 entries, 0 to 1230
Data columns (total 97 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   barcode                                    1231 non-null   object 
 1   patient                                    1231 non-null   object 
 2   sample                                     1231 non-null   object 
 3   shortLetterCode                            1231 non-null   object 
 4   definition                                 1231 non-null   object 
 5   sample_submitter_id                        1231 non-null   object 
 6   sample_type_id                             1231 non-null   int64  
 7   tumor_descriptor                           1231 non-null   object 
 8   sample_id                                  1231 non-null   object 
 9   submitter_id                               1231 non-null   object 
 10  sa

In [20]:
# Summary Statistics
print("\nSummary Statistics:")
display(info_df.describe())
display(assay_df.describe())


Summary Statistics:


,sample_type_id,days_to_collection,initial_weight,days_to_diagnosis,treatments,age_at_diagnosis,year_of_diagnosis,sites_of_involvement,age_at_index,days_to_birth,...,days_to_last_follow_up,year_of_birth,paper_days_to_birth,paper_days_to_death,paper_days_to_last_followup,paper_age_at_initial_pathologic_diagnosis,paper_Tumor_Grade,paper_MSI_status,paper_HPV_Status,paper_tobacco_smoking_history
count,1231.000000,1229.000000,1216.000000,1190.000000,0.0,1176.000000,1110.000000,0.0,1230.000000,1214.000000,...,3.0,3.0,1084.000000,152.000000,995.000000,1099.000000,0.0,0.0,0.0,0.0
mean,1.946385,1130.716843,310.907072,70.116807,NaN,21530.009354,2007.763063,NaN,58.280488,-21524.601318,...,906.0,1946.0,-21572.763838,1639.546053,1175.024121,58.424932,NaN,NaN,NaN,NaN
std,2.903945,1495.584209,271.972415,359.140965,NaN,4815.422701,4.256506,NaN,13.277725,4842.015435,...,0.0,0.0,4788.797180,1312.563075,1167.812152,13.146040,NaN,NaN,NaN,NaN
min,1.000000,16.000000,5.000000,0.000000,NaN,9840.000000,1988.000000,NaN,26.000000,-32872.000000,...,906.0,1946.0,-32873.000000,116.000000,-31.000000,26.000000,NaN,NaN,NaN,NaN
25%,1.000000,158.000000,130.000000,0.000000,NaN,17959.250000,2007.000000,NaN,48.000000,-24822.750000,...,906.0,1946.0,-24843.250000,668.250000,436.000000,49.000000,NaN,NaN,NaN,NaN
50%,1.000000,484.000000,220.000000,0.000000,NaN,21472.000000,2009.000000,NaN,58.000000,-21494.500000,...,906.0,1946.0,-21557.000000,1223.000000,761.000000,58.000000,NaN,NaN,NaN,NaN
75%,1.000000,1407.000000,392.500000,0.000000,NaN,24788.750000,2010.000000,NaN,67.000000,-17873.000000,...,906.0,1946.0,-18019.250000,2364.000000,1545.500000,67.000000,NaN,NaN,NaN,NaN
max,11.000000,7858.000000,2190.000000,4343.000000,NaN,32872.000000,2013.000000,NaN,89.000000,-9706.000000,...,906.0,1946.0,-9706.000000,7455.000000,8605.000000,90.000000,NaN,NaN,NaN,NaN


,TCGA-A2-A25D-01A-12R-A16F-07,TCGA-BH-A201-01A-11R-A14M-07,TCGA-AC-A23C-01A-12R-A169-07,TCGA-AR-A5QP-01A-11R-A28M-07,TCGA-C8-A12P-01A-11R-A115-07,TCGA-BH-A0W3-01A-11R-A109-07,TCGA-AC-A8OQ-01A-11R-A41B-07,TCGA-BH-A0H6-01A-21R-A056-07,TCGA-D8-A27V-01A-12R-A17B-07,TCGA-D8-A27M-01A-11R-A16F-07,...,TCGA-AC-A3TM-01A-11R-A22K-07,TCGA-A7-A26J-01A-11R-A277-07,TCGA-LL-A441-01A-11R-A24H-07,TCGA-A7-A13G-01A-11R-A13Q-07,TCGA-BH-A0E6-01A-11R-A034-07,TCGA-LL-A5YP-01A-21R-A28M-07,TCGA-AO-A03L-01A-41R-A056-07,TCGA-BH-A42T-01A-11R-A24H-07,TCGA-A2-A04W-01A-31R-A115-07,TCGA-AR-A24O-01A-11R-A169-07
count,6.066000e+04,60660.000000,60660.000000,60660.000000,60660.000000,60660.000000,6.066000e+04,60660.000000,60660.000000,60660.000000,...,60660.000000,6.066000e+04,60660.000000,6.066000e+04,60660.000000,60660.000000,60660.000000,60660.000000,60660.000000,60660.000000
mean,1.034877e+03,1110.244313,1202.395219,783.117441,916.485295,661.606116,7.279932e+02,818.576805,1603.021348,1159.860320,...,802.418299,7.182427e+02,1039.272107,1.128893e+03,858.408342,743.927052,990.667672,1062.995434,749.378932,1016.379080
std,1.105579e+04,9046.044855,9528.579238,5482.329634,8646.343421,3895.447378,1.274565e+04,7125.083981,11491.685458,8436.901089,...,5758.350828,1.312690e+04,6726.086821,2.792600e+04,9505.848762,5561.210350,6703.815993,7844.225323,8891.101552,8029.830758
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000e+00,1.000000,2.000000,1.000000,0.000000,0.000000,1.000000e+00,1.000000,1.000000,2.000000,...,1.000000,1.000000e+00,2.000000,5.000000e+00,1.000000,1.000000,1.000000,1.000000,0.000000,2.000000
75%,5.600000e+01,82.000000,103.000000,71.000000,44.000000,37.000000,7.000000e+01,52.000000,96.000000,90.000000,...,62.000000,5.000000e+01,131.000000,1.060000e+02,58.000000,50.000000,55.000000,70.000000,36.000000,82.000000
max,1.188852e+06,838143.000000,885193.000000,494952.000000,742211.000000,422219.000000,2.406507e+06,722441.000000,969281.000000,739509.000000,...,365523.000000,3.005554e+06,563087.000000,5.519593e+06,935446.000000,564015.000000,490954.000000,550624.000000,865035.000000,771351.000000


In [21]:
assay_df_transposed = assay_df.T.reset_index().rename(columns={'index': 'barcode'})
merged_df = pd.merge(info_df, assay_df_transposed, on='barcode', how='inner')

print("\nMerged Dataset Information:")
print(merged_df.head())
print(merged_df.shape)


Merged Dataset Information:
                        barcode       patient            sample  \
0  TCGA-A2-A25D-01A-12R-A16F-07  TCGA-A2-A25D  TCGA-A2-A25D-01A   
1  TCGA-BH-A201-01A-11R-A14M-07  TCGA-BH-A201  TCGA-BH-A201-01A   
2  TCGA-AC-A23C-01A-12R-A169-07  TCGA-AC-A23C  TCGA-AC-A23C-01A   
3  TCGA-AR-A5QP-01A-11R-A28M-07  TCGA-AR-A5QP  TCGA-AR-A5QP-01A   
4  TCGA-C8-A12P-01A-11R-A115-07  TCGA-C8-A12P  TCGA-C8-A12P-01A   

  shortLetterCode           definition sample_submitter_id  sample_type_id  \
0              TP  Primary solid Tumor    TCGA-A2-A25D-01A               1   
1              TP  Primary solid Tumor    TCGA-BH-A201-01A               1   
2              TP  Primary solid Tumor    TCGA-AC-A23C-01A               1   
3              TP  Primary solid Tumor    TCGA-AR-A5QP-01A               1   
4              TP  Primary solid Tumor    TCGA-C8-A12P-01A               1   

  tumor_descriptor                             sample_id  submitter_id  ...  \
0          Primary  

In [17]:
# Check for missing values
print(merged_df.isnull().sum())

# Check for duplicate rows
duplicate_rows = merged_df[merged_df.duplicated(keep=False)]
print(f"Number of duplicate rows: {duplicate_rows.shape[0]}")

barcode            0
patient            0
sample             0
shortLetterCode    0
definition         0
                  ..
60655              0
60656              0
60657              0
60658              0
60659              0
Length: 60757, dtype: int64
Number of duplicate rows: 0


## 2.2 Feature Description Table <a name="2.2"></a>

**No** | **Feature Name**                             | **Description**                                        | **Data Type**   | **Data Classification** |
-------|----------------------------------------------|--------------------------------------------------------|----------------|--------------------------|
|1| barcode                                      | Unique barcode for each sample                        | String         | Categorical              |
|2| patient                                      | Unique patient identifier                             | String         | Categorical              |
|3| sample                                       | Sample identifier                                     | String         | Categorical              |
|4| shortLetterCode                              | Sample short code classification                      | String         | Categorical              |
|5| definition                                   | Definition of the sample type                         | String         | Categorical              |
|6| sample_submitter_id                          | Submitter assigned sample ID                         | String         | Categorical              |
|7| sample_type_id                               | Numeric sample type identifier                        | Integer        | Categorical              |
|8| tumor_descriptor                             | Description of tumor type                            | String         | Categorical              |
|9| sample_id                                    | Sample unique identifier                             | String         | Categorical              |
|10| submitter_id                                 | Submitter assigned ID                                | String         | Categorical              |
|11| sample_type                                  | Sample classification (Tumor/Normal)                | String         | Categorical              |
|12| oct_embedded                                 | If sample was OCT embedded                           | Boolean        | Categorical              |
|13| specimen_type                                | Type of specimen collected                          | String         | Categorical              |
|14| state                                        | Current sample state                                | String         | Categorical              |
|15| is_ffpe                                      | Formalin-Fixed Paraffin-Embedded status             | Boolean        | Categorical              |
|16| tissue_type                                  | Type of tissue collected                            | String         | Categorical              |
|17| preservation_method                          | Sample preservation technique                      | String         | Categorical              |
|18| composition                                  | Composition of collected sample                     | String         | Categorical              |
|19| days_to_collection                           | Days until sample collection                        | Float          | Numerical                |
|20| initial_weight                               | Initial weight of the sample (if recorded)         | Float          | Numerical                |
|21| pathology_report_uuid                        | Unique identifier for pathology report              | String         | Categorical              |
|22| synchronous_malignancy                       | Presence of synchronous malignancy                 | String         | Categorical              |
|23| ajcc_pathologic_stage                        | AJCC pathologic stage                              | String         | Categorical              |
|24| days_to_diagnosis                            | Days between diagnosis and data collection         | Float          | Numerical                |
|25| laterality                                   | Laterality of tumor (Left/Right)                   | String         | Categorical              |
|26| treatments                                   | Treatment details                                  | Float          | Numerical                |
|27| tissue_or_organ_of_origin                    | Origin of the tissue or organ                      | String         | Categorical              |
|28| age_at_diagnosis                             | Age at diagnosis                                   | Float          | Numerical                |
|29| primary_diagnosis                            | Primary diagnosis                                 | String         | Categorical              |
|30| metastasis_at_diagnosis                      | Presence of metastasis at diagnosis               | String         | Categorical              |
|31| prior_malignancy                             | History of prior malignancies                     | String         | Categorical              |
|32| year_of_diagnosis                            | Year of diagnosis                                 | Float          | Numerical                |
|33| prior_treatment                              | Whether prior treatment was given                 | String         | Categorical              |
|34| diagnosis_is_primary_disease                 | If diagnosis is for primary disease               | String         | Categorical              |
|35| method_of_diagnosis                          | Diagnostic method used                            | String         | Categorical              |
|36| ajcc_staging_system_edition                  | Edition of AJCC staging system                   | String         | Categorical              |
|37| ajcc_pathologic_t                            | AJCC pathologic T (Tumor size classification)    | String         | Categorical              |
|38| morphology                                   | Tumor morphology classification                  | String         | Categorical              |
|39| ajcc_pathologic_n                            | AJCC pathologic N (Lymph node classification)    | String         | Categorical              |
|40| ajcc_pathologic_m                            | AJCC pathologic M (Metastasis classification)    | String         | Categorical              |
|41| classification_of_tumor                      | Classification of the tumor                      | String         | Categorical              |
|42| diagnosis_id                                 | Unique identifier for diagnosis                  | String         | Categorical              |
|43| icd_10_code                                  | ICD-10 classification code                       | String         | Categorical              |
|44| site_of_resection_or_biopsy                  | Location of resection or biopsy                  | String         | Categorical              |
|45| sites_of_involvement                         | Sites where metastasis occurred                  | Float          | Numerical                |
|46| race                                         | Patient’s race                                   | String         | Categorical              |
|47| gender                                       | Patient’s gender                                | String         | Categorical              |
|48| ethnicity                                    | Patient’s ethnicity                             | String         | Categorical              |
|49| vital_status                                 | Patient’s vital status (Alive/Deceased)         | String         | Categorical              |
|50| age_at_index                                 | Age at index date                               | Float          | Numerical                |
|51| days_to_birth                                | Number of days from birth to index date         | Float          | Numerical                |
|52| demographic_id                               | Unique identifier for demographic details       | String         | Categorical              |
|53| age_is_obfuscated                            | If patient’s age is obfuscated                  | String         | Categorical              |
|54| country_of_residence_at_enrollment           | Country of residence during enrollment          | String         | Categorical              |
|55| bcr_patient_barcode                          | Barcode for patient identification              | String         | Categorical              |
|56| primary_site                                 | Primary Site                                    | Float          | Numerical                | 
|57| project_id                                   | Project ID related to study                     | String         | Categorical              |
|58| disease_type                                 | Disease Type                                    | float64        | Numerical                |
|59| name                                         | Name of the dataset sample                      | String         | Categorical              |
|60| releasable                                   | If data is releasable                           | Boolean        | Categorical              |
|61| released                                     | If data has been released                       | Boolean        | Categorical              |
|62| days_to_death                                | Days from diagnosis to death                    | Float          | Numerical                |
|63| tumor_of_origin                              | Origin of the tumor                            | String         | Categorical              |
|64| figo_stage                                   | FIGO staging classification                    | String         | Categorical              |
|65| figo_staging_edition_year                    | Year of FIGO staging edition                   | Float          | Numerical                |
|66| days_to_sample_procurement                   | Days from diagnosis to sample procurement      | Float          | Numerical                |
|67| last_known_disease_status                    | Last known disease status                      | String         | Categorical              |
|68| days_to_last_follow_up                       | Days from last follow-up                       | Float          | Numerical                |
|69| tumor_grade                                  | Tumor grade classification                     | String         | Categorical              |
|70| progression_or_recurrence                    | If cancer has progressed or recurred           | String         | Categorical              |
|71| alcohol_history                              | Patient's alcohol consumption history         | String         | Categorical              |
|72| exposure_id                                  | Unique ID for exposure details                 | String         | Categorical              |
|73| year_of_birth                                | Patient’s year of birth                        | Float          | Numerical                |
|74| paper_patient                               | Unique patient identifier used in study               | String         | Categorical              |
|75| paper_Tumor.Type                            | Tumor type classification from study                 | String         | Categorical              |
|76| paper_Included_in_previous_marker_papers    | Indicates if sample was included in previous studies | String         | Categorical              |
|77| paper_vital_status                          | Patient's vital status (Alive/Deceased)              | String         | Categorical              |
|78| paper_days_to_birth                         | Number of days from birth to study date              | Float          | Numerical                |
|79| paper_days_to_death                         | Number of days from diagnosis to death               | Float          | Numerical                |
|80| paper_days_to_last_followup                 | Days since last follow-up                            | Float          | Numerical                |
|81| paper_age_at_initial_pathologic_diagnosis   | Age at initial pathologic diagnosis                  | Float          | Numerical                |
|82| paper_pathologic_stage                      | Pathologic stage classification                      | String         | Categorical              |
|83| paper_Tumor_Grade                           | Tumor grade classification                          | Float          | Numerical                |
|84| paper_BRCA_Pathology                        | BRCA pathology classification                       | String         | Categorical              |
|85| paper_BRCA_Subtype_PAM50                    | BRCA subtype classification using PAM50 method      | String         | Categorical              |
|86| paper_MSI_status                            | Microsatellite instability (MSI) status             | Float          | Numerical                |
|87| paper_HPV_Status                            | HPV infection status                                | Float          | Numerical                |
|88| paper_tobacco_smoking_history               | History of tobacco smoking                          | Float          | Numerical                |
|89| paper_CNV.Clusters                          | Copy number variation (CNV) clusters                | String         | Categorical              |
|90| paper_Mutation.Clusters                     | Mutation-based clustering classification           | String         | Categorical              |
|91| paper_DNA.Methylation.Clusters              | DNA methylation clustering classification          | String         | Categorical              |
|92| paper_mRNA.Clusters                         | mRNA-based clustering classification               | String         | Categorical              |
|93| paper_miRNA.Clusters                        | miRNA-based clustering classification              | String         | Categorical              |
|94| paper_lncRNA.Clusters                       | lncRNA-based clustering classification             | String         | Categorical              |
|95| paper_Protein.Clusters                      | Protein-based clustering classification            | String         | Categorical              |
|96| paper_PARADIGM.Clusters                     | Pathway-based clustering using PARADIGM            | String         | Categorical              |
|97| paper_Pan.Gyn.Clusters                      | Pan-Gynecologic clustering classification         | String         | Categorical              |


# 3. NORMALIZATION <a name="3"></a>

# 4. EXPLORATORY DATA ANALYSIS (EDA) <a name="4"></a>